# Summary Outputs

When you run a simulation in PassengerSim, you will get a `SimulationTables` object.
This objects embeds a variety of summary infomation from the run.

In [ ]:
import passengersim as pax

pax.versions()

In [ ]:
cfg = pax.Config.from_yaml(pax.demo_network("3MKT/08-untrunc-em"))
cfg.simulation_controls.num_samples = 300
cfg.simulation_controls.num_trials = 2
sim = pax.Simulation(cfg)
summary = sim.run()

The simple `repr` of this object has a bit of information about what data is in there.
You can view this in a Jupyter notebook by putting the object as the last line of a cell
(or just by itself):

In [ ]:
summary

We can see here there are a variety of tables stored as pandas DataFrames. We
can access the raw values of any of these dataframes directly in Python as an
attribute on the `SimulationTables` object.

In [ ]:
summary.carriers

There is also some metadata stored on the summary, which can be accessed via the `metadata` method.

In [ ]:
summary.metadata()

You can access the metadata for a specific key by passing that key as an argument to the method.

In [ ]:
summary.metadata("time")

In [ ]:
assert "created" in summary.metadata("time").keys()
assert "time.created" in summary.metadata().keys()

## Saving and Restoring

Running a PassengerSim simulation on a practical network can take some time, 
so it is desirable to save your results after a run.  This allows you to come
back to the analyze those results later, or compare against other future scenario
permutations.  Saving outputs is done efficiently in the `pickle` format by
using the `to_pickle` method.  This will automatically also use LZ4 compression 
if available, which will reduce the filesize considerably.

In [ ]:
# ensure the saved-outputs directory starts off empty
import shutil

shutil.rmtree("saved-outputs", ignore_errors=True)

In [ ]:
summary.to_pickle("saved-outputs/summary")

The method will automatically add the appropriate file extension and write the file to disk.
By default, it will also add a timestamp, so that you will not overwrite existing similar
files.

In [ ]:
from passengersim.utils.show_dir import display_directory_contents

display_directory_contents("saved-outputs")

Restoring from this pickle file can be done, surprisingly enough, using the 
`from_pickle` method.  You can call this method with the same filename as
`to_pickle`, and it will load the file with the most recent timestamp if there
is one or more matching file(s) with various timestamps.  To load a specific
pickle file that may or may not be the most recent, you can call this method
with the complete actual filename, including the timestamp and ".pkl" or 
".pkl.lz4" suffix.

In [ ]:
resummary = pax.SimulationTables.from_pickle("saved-outputs/summary")
resummary

Once loaded, you can confirm which file was loaded, as that is added to the metadata at load time:

In [ ]:
resummary.metadata("loaded")

### Lazy Data Access

When it is time to load the data, the pickle format is an all-or-nothing file format: 
if you only need a tiny bit of information from the file (e.g. what was AL1's revenue?)
you still need to read *all* the data in the file: legs, paths, etc. PassengerSim also 
includes an alternative file format that stores the various data tables from the 
`SimulationTables` in independently loadable sections.  This format otherwise works the
same as the pickle file, but uses `to_file` and `from_file`, and it creates files with
a "*.pxsim" extension.

In [ ]:
summary.to_file("saved-outputs/summary")

display_directory_contents("saved-outputs")

When opening the file, only the most basic metadata is loaded by the `from_file` method,
and the rest is identified as available on demand from storage.

In [ ]:
resummary2 = pax.SimulationTables.from_file("saved-outputs/summary")
resummary2

The metadata about this file is available under the "store" key instead of "loaded".

In [ ]:
resummary2.metadata("store")

Accessing data will load just that table from the file.  this includes accessing a table
explicity (by calling for it), or implicitly (e.g. by creating a figure using the data).

In [ ]:
resummary2.carriers

In [ ]:
resummary2.fig_fare_class_mix()

We can see in the objects `repr` that the carriers and fare_class_mix tables have been loaded,
but the rest are still only in the storage file.

In [ ]:
resummary2